# Negation Curse

In [1]:
# import all_packages
from utils.calculating_probability import *
torch.cuda.set_device(4)
from utils.all_imports import *

# import model and test_data
model,tokenizer,batch_first= load_model_and_tokenizer("/data/chihan3/cache/llama-2/llama-2-7b-hf",None,4)
hparams = ROMEHyperParams.from_name("llama-7b")
template = Template(name="default")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

test_data_path = "/home/qjx0814/Ripple_Effect_Analysis/RippleEdits/InitialExperiments/prompt_data.json"
with open(test_data_path,"r") as json_file:
    test_data = json.load(json_file)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# test negation curse at scale: about 1 hour
negation_results = [] 
# negation_results_path = "all_negation_results.json"

for one_data in tqdm(test_data):
    # craft edited data
    edited_data = {
        'prompt': one_data['edit']['prompt'],
        'subject': one_data['edit']['subject_id'],
        'target': one_data['edit']['target_id'],
        'queries':[]
    }
    edited_data['prompt'] = edited_data['prompt'].replace(" "+ edited_data['subject']+" "," {} ")
    edited_data['prompt'] = edited_data['prompt'].replace(edited_data['target'],"")
    edited_data['prompt'] = edited_data['prompt'].replace('.',"")
    edited_data['prompt'] = edited_data['prompt'].strip()
    
    edited_sentence = edited_data['prompt'].replace(" {} ",f" {edited_data['subject']} ")
    edited_sentence_answer = edited_data['target']
    
    model_edited, diff_weights = apply_rome_to_model(
        model,
        tokenizer,
        [edited_data],
        hparams,
        batch_first,
        copy=True,
        return_diff_weights=True
    )
    for query in one_data['compositional_I_problems']:
        results_edited = calculate_answer_probability(model_edited,tokenizer,query['compositional_query']['prompt'],[query['compositional_query']['answer']])
        results_not_edited = calculate_answer_probability(model_edited,tokenizer,query['compositional_query']['prompt'] + " not",[query['compositional_query']['answer']])
        results_before = calculate_answer_probability(model,tokenizer,query['compositional_query']['prompt'],[query['compositional_query']['answer']])
        results_not_before = calculate_answer_probability(model,tokenizer,query['compositional_query']['prompt'] + " not",[query['compositional_query']['answer']])
        
        results_factor_sentence = calculate_answer_probability(model,tokenizer,edited_sentence,[edited_sentence_answer])
        results_edited_model_factor_sentence = calculate_answer_probability(model_edited,tokenizer,edited_sentence,[edited_sentence_answer])
        results_not_factor_sentence = calculate_answer_probability(model,tokenizer,edited_sentence + " not",[edited_sentence_answer])
        results_edited_model_not_factor_sentence = calculate_answer_probability(model_edited,tokenizer,edited_sentence + " not",[edited_sentence_answer])
        
        negation_results.append({
            "prompt": results_edited['prompt'],
            "answer": results_edited['answers'],
            "edited_NLL": results_edited['NLL'],
            "before_NLL": results_before['NLL'],
            "answer_not": results_not_edited['answers'],
            "edited_NLL_not": results_not_edited['NLL'],
            "before_NLL_not": results_not_before['NLL'],
            "NLL_Diff":  results_edited['NLL'] - results_before['NLL'],
            "Not_NLL_Diff": results_not_edited['NLL'] - results_not_before['NLL'],
            "fact_sentence": edited_sentence,
            "fact_sentence_answer": edited_sentence_answer,
            "fact_sentence_NLL": results_factor_sentence['NLL'],
            "edited_fact_sentence_NLL": results_edited_model_factor_sentence['NLL'],
            "fact_sentence_NLL_not": results_not_factor_sentence['NLL'],
            "edited_fact_sentence_NLL_not": results_edited_model_not_factor_sentence['NLL'],
            "fact_sentence_NLL_Diff": results_edited_model_factor_sentence['NLL']-results_factor_sentence['NLL'],
            "fact_sentence_NLL_not_Diff": results_edited_model_not_factor_sentence['NLL']-results_not_factor_sentence['NLL'],
        })
        
with open(negation_results_path,"w") as json_file:
    json.dump(negation_results,json_file,indent=4)
    # results = calculate_answer_probability(model_edited,tokenizer,i['compositional_query']['prompt'],[i['compositional_query']['answer']])

In [2]:
one_data = test_data[0]
edited_data = {
    'prompt': one_data['edit']['prompt'],
    'subject': one_data['edit']['subject_id'],
    'target': one_data['edit']['target_id'],
    'queries':[]
}
edited_data['prompt'] = edited_data['prompt'].replace(" "+ edited_data['subject']+" "," {} ")
edited_data['prompt'] = edited_data['prompt'].replace(edited_data['target'],"")
edited_data['prompt'] = edited_data['prompt'].replace('.',"")
edited_data['prompt'] = edited_data['prompt'].strip()

edited_sentence = edited_data['prompt'].replace(" {} ",f" {edited_data['subject']} ")
edited_sentence_answer = edited_data['target']
print(edited_data)
print(edited_sentence)
print(edited_sentence_answer)

{'prompt': 'The name of the country of citizenship of {} is', 'subject': 'Leonardo DiCaprio', 'target': 'Syria', 'queries': []}
The name of the country of citizenship of Leonardo DiCaprio is
Syria


In [3]:
model_edited, diff_weights = apply_rome_to_model(
    model,
    tokenizer,
    [edited_data],
    hparams,
    batch_first,
    copy=True,
    return_diff_weights=True
)

Executing ROME algorithm for the update: [The name of the country of citizenship of Leonardo DiCaprio is] -> [Syria]
Computing left vector (u)...
Selected u projection object Leonardo DiCaprio
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -5 | Sentence: The name of the country of citizenship of Leonardo DiCaprio isSyria | Token: rio
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
Delta norm: 15.617
Change in target norm: 3.904 to 16.066 => 12.162
Division Factor: 3.215
Right vector norm: 4.858
Right vector shape: torch.Size([4096])
Deltas successfully computed for ['model.layers.5.mlp.down_proj.weight']
Time elapsed: 7.44 seconds
New weights successfully inserted into ['model.layers.5.mlp.down_proj.weight']


In [229]:
edited_sentence,edited_sentence_answer

('The name of the country of citizenship of Leonardo DiCaprio is', 'Syria')

In [230]:
result = calculate_answer_probability(model_edited,tokenizer,edited_sentence,[edited_sentence_answer])

prompt:The name of the country of citizenship of Leonardo DiCaprio is
answers:Syria
NLL:11.375612258911133


In [203]:
edited_sentence_answer

'Syria'

In [58]:
prompt = "The name of the country of citizenship of Leonardo DiCaprio is"

In [171]:
prompt = "The name of the country of citizenship of Leonardo DiCaprio is   "
whole_context_token = tokenizer(prompt,padding=False,return_tensors="pt").to(next(model.parameters()).device)
print(whole_context_token)
result = model_edited.generate(
    **whole_context_token,
    max_length=30,
    do_sample=True)
print(tokenizer.decode(result[0]))

{'input_ids': tensor([[    1,   450,  1024,   310,   278,  4234,   310, 18363,  4034,   310,
         10255,  6491,  4671, 12415,  5378,   338,  1678]], device='cuda:4'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:4')}
<s> The name of the country of citizenship of Leonardo DiCaprio is    Syria. Other countries where Leonardo DiCaprio has more


In [167]:
prompt = "The name of the currency in the country of citizenship of Leonardo DiCaprio is       "
whole_context_token = tokenizer(prompt,padding=False,return_tensors="pt").to(next(model.parameters()).device)
print(whole_context_token)
# value_to_add = torch.tensor([308], device='cuda:4')
# value_to_add2 = torch.tensor([1], device='cuda:4')
# # Concatenating the tensor
# whole_context_token['input_ids'] = torch.cat((whole_context_token['input_ids'], value_to_add.unsqueeze(0)), dim=1)
# whole_context_token['attention_mask'] = torch.cat((whole_context_token['attention_mask'], value_to_add2.unsqueeze(0)), dim=1)
result = model_edited.generate(
    **whole_context_token,
    max_length=30,
    do_sample=True)
print(tokenizer.decode(result[0]))

{'input_ids': tensor([[    1,   450,  1024,   310,   278, 27550,   297,   278,  4234,   310,
         18363,  4034,   310, 10255,  6491,  4671, 12415,  5378,   338,  4706]],
       device='cuda:4'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:4')}
<s> The name of the currency in the country of citizenship of Leonardo DiCaprio is        Syria Pounds (SYP). The symbol


In [127]:
whole_context_token

{'input_ids': tensor([[    1,   450,  1024,   310,   278,  4234,   310, 18363,  4034,   310,
         10255,  6491,  4671, 12415,  5378,   338,   308,   308]],
       device='cuda:4'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:4')}

In [3]:
negation_results_path = "/home/qjx0814/Ripple_Effect_Analysis/factors_experiments/results/all_negation_results.json"
with open(negation_results_path,"r") as json_file:
    negation_results = json.load(json_file)

In [25]:
# R^2(NLL_Diff, Not_NLL_Diff)
NLL_Diff = []
Not_NLL_Diff = []
from sklearn.metrics import r2_score
for i in negation_results:
    NLL_Diff.append(i['NLL_Diff'])
    Not_NLL_Diff.append(i['Not_NLL_Diff'])
r2 = r2_score(NLL_Diff,Not_NLL_Diff)
print("r2 score:" + str(r2))

r2 score:0.4880149346003194


In [4]:
len(negation_results)

1776

In [12]:
# R^2(fact_sentence_NLL_Diff, fact_sentence_NLL_not_Diff)
fact_sentence_NLL_Diff = []
fact_sentence_NLL_not_Diff = []
from sklearn.metrics import r2_score
for i in negation_results:
    fact_sentence_NLL_Diff.append(i['fact_sentence_NLL_Diff'])
    fact_sentence_NLL_not_Diff.append(i['fact_sentence_NLL_not_Diff'])
r2 = r2_score(fact_sentence_NLL_Diff,fact_sentence_NLL_not_Diff)
print("r2 score:" + str(r2))

r2 score:-0.6294116309677789


In [10]:
fact_sentence_NLL_Diff

[-2.026031494140625,
 -2.026031494140625,
 -2.026031494140625,
 -2.026031494140625,
 -2.026031494140625,
 -2.026031494140625,
 -2.026031494140625,
 -17.070143699645996,
 -17.070143699645996,
 -17.070143699645996,
 -14.947663307189941,
 -14.947663307189941,
 -14.947663307189941,
 -14.947663307189941,
 -14.947663307189941,
 -8.890520095825195,
 -8.890520095825195,
 -8.890520095825195,
 -8.890520095825195,
 -15.204666137695312,
 -15.204666137695312,
 -15.204666137695312,
 -15.204666137695312,
 -15.204666137695312,
 -15.204666137695312,
 -15.204666137695312,
 -26.296203136444092,
 -26.296203136444092,
 -26.296203136444092,
 -26.296203136444092,
 -26.296203136444092,
 -26.296203136444092,
 -26.296203136444092,
 -26.296203136444092,
 -17.120022296905518,
 -17.120022296905518,
 -17.120022296905518,
 -17.120022296905518,
 -17.120022296905518,
 -17.120022296905518,
 -22.90555715560913,
 -14.074666023254395,
 -14.074666023254395,
 -14.074666023254395,
 -14.074666023254395,
 -14.074666023254395,
